In [ ]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import numpy as np


In [ ]:
df=pd.read_excel("Input.xlsx")

In [ ]:
df=df.iloc[0:150]

In [ ]:
df

In [ ]:

df.drop('URL_ID',axis=1,inplace=True)

# Data Extraction

In [ ]:
#extracting text from all the url
url_id=37
for i in range(0,len(df)):
    
 
    link=df.iloc[i].values
   
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    } #giving user access
    req=requests.get(link[0],headers=headers)
    soup=BeautifulSoup(req.content,'html.parser')
    content=soup.find_all(attrs={'class':'td-post-content'}) or soup.find_all(attrs={'class':"td-404-sub-title"})
    content=content[0].text.replace('\xa0',"  ").replace('\n'," ")
    title=soup.find_all(attrs={'class':'entry-title'}) 
    title=title[16].text.replace('\n',"  ").replace('/',"")
    text=title+ '.'+content
    text=np.array(text)
    text.reshape(1,-1)
    df1=pd.Series(text)
    filename=str(url_id)+'.'+'txt'
    with open(filename,'a',encoding='utf-8') as f:
        df1.to_csv(f,line_terminator=',',index=False,header=False)
        #files.download(filename)
    url_id+=1    
 
    
    


# Data Analysis

In [ ]:
text=pd.read_csv('150.txt',header=None)

In [ ]:
text.info()

In [ ]:
text

In [ ]:
text.drop(1,axis=1,inplace=True)

In [ ]:
text=text.astype(str)

In [ ]:
import re

a=text[0].str.split('([\.]\s)',expand=False).explode()
b=pd.DataFrame(a)
b.columns=['abc']

In [ ]:
def char_remove(x):
    nopunc=[char for char in x if char!='.']
    return''.join(nopunc)
b['abc']=b['abc'].apply(char_remove)

In [ ]:
null=b.replace('',np.nan,regex=True)
null=null.mask(null==" ")
null=null.dropna()
null.reset_index(drop=True,inplace=True)

In [ ]:
null

In [ ]:
import nltk
import string

In [ ]:
punc=[punc for punc in string.punctuation]
punc

In [ ]:
StopWords_Auditor=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_Auditor.txt',header=None)
StopWords_Currencies=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_Currencies.txt',header=None,sep='|',encoding='latin-1')
StopWords_DatesandNumbers=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_DatesandNumbers.txt',header=None)
StopWords_Generic=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_Generic.txt',header=None)
StopWords_GenericLong=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_GenericLong.txt',header=None)
StopWords_Geographic=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_Geographic.txt',header=None)
StopWords_Names=pd.read_csv('C:/Users/abc/Project/StopWords/StopWords_Names.txt',header=None)

In [ ]:
def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])


In [ ]:
null['abc']=null['abc'].apply(text_process)
null

In [ ]:
positive=pd.read_csv('C:/Users/abc/Project/MasterDictionary/negative-words.txt',header=None,encoding='latin-1')
negative=pd.read_csv('C:/Users/abc/Project/MasterDictionary/positive-words.txt',header=None,encoding='latin-1')

In [ ]:
positive.columns=['abc']
negative.columns=['abc']
positive['abc']=positive['abc'].astype(str)
negative['abc']=negative['abc'].astype(str)

In [ ]:
positive['abc']=positive['abc'].apply(text_process)
negative['abc']=negative['abc'].apply(text_process)

In [ ]:
#positive list
length=positive.shape[0]
post=[]
for i in range(0,length):
    nopunc =[char for char in positive.iloc[i] if char not in string.punctuation or char != '+']
    nopunc=''.join(nopunc)

    post.append(nopunc)

In [ ]:
#negative list
length=negative.shape[0]
neg=[]
for i in range(0,length):
    nopunc =[char for char in negative.iloc[i] if char not in string.punctuation or char != '+']
    nopunc=''.join(nopunc)
    neg.append(nopunc)

In [ ]:
#importing tokenize library
from nltk.tokenize import word_tokenize

In [ ]:
txt_list=[]
length=null.shape[0]
for i in range(0,length):
    txt=' '.join([word for word in null.iloc[i]])
    txt_list.append(txt)

In [ ]:
nltk.download('punkt')

In [ ]:
#tokenization of text
tokenize_text=[]
for i in txt_list:
    tokenize_text+=(word_tokenize(i))

In [ ]:
tokenize_text

In [ ]:
len(tokenize_text)

# POSITIVE SCORE

In [ ]:
positive_score=0
for i in tokenize_text:
    if(i.lower() in post):
        positive_score+=1
print('postive score=', positive_score)

# NEGATIVE SCORE

In [ ]:
negative_score=0
for i in tokenize_text:
    if(i.lower() in neg):
        negative_score+=1
print('negative score=', negative_score)

# POLARITY SCORE

In [ ]:
#Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
Polarity_Score=(positive_score-negative_score)/((positive_score+negative_score)+0.000001)
print('polarity_score=', Polarity_Score)

# SUBJECTIVITY SCORE

In [ ]:
#Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
subjectiivity_score=(positive_score-negative_score)/((len(tokenize_text))+ 0.000001)
print('subjectivity_score',subjectiivity_score)

# AVG. SENTENCE LENGTH

In [ ]:
length=null.shape[0]
avg_length=[]
for i in range(0,length):
    avg_length.append(len(null['abc'].iloc[i]))
avg_senetence_length=sum(avg_length)/len(avg_length)
print('avg sentence length=', avg_senetence_length)

# PERCENTAGE OF COMPLEX WORDS

In [ ]:
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
    x=re.compile('[es|ed]$')
    if x.match(i.lower()):
        count+=0
    else:
        for j in i:
            if(j.lower() in vowels ):
                count+=1
    if(count>2):
        complex_Word_Count+=1
    count=0


In [ ]:
Percentage_of_Complex_words=complex_Word_Count/len(tokenize_text)
print('percentag of complex words= ',Percentage_of_Complex_words)

# FOG INDEX

In [ ]:
#Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
Fog_Index = 0.4 * (avg_senetence_length + Percentage_of_Complex_words)
print('fog index= ',Fog_Index )

# AVG. NUMBER OF EORDS PER SENTENCE

In [ ]:
length=null.shape[0]
avg_length=[]
for i in range(0,length):
    a=[word.split( ) for word in null.iloc[i]]
    avg_length.append(len(a[0]))
    a=0
#avg
avg_no_of_words_per_sentence=sum(avg_length)/length
print("avg no of words per sentence= ",avg_no_of_words_per_sentence)

# COMPLEX WORD COUNT

In [ ]:
vowels=['a','e','i','o','u']
import re
count=0
complex_Word_Count=0
for i in tokenize_text:
    x=re.compile('[es|ed]$')
    if x.match(i.lower()):
        count+=0
    else:
        for j in i:
            if(j.lower() in vowels ):
                count+=1
    if(count>2):
        complex_Word_Count+=1
    count=0
print('complex words count=',  complex_Word_Count)

# WORD COUNT

In [ ]:
word_count=len(tokenize_text)
print('word count= ', word_count)

# SYLLABLE PER WORD

In [ ]:
vowels=['a','e','i','o','u']
import re
count=0
for i in tokenize_text:
    x=re.compile('[es|ed]$')
    if x.match(i.lower()):
        count+=0
    else:
        for j in i:
            if(j.lower() in vowels ):
                count+=1
syllable_count=count
print('syllable_per_word= ',syllable_count)

# PERSONAL PRONOUNS

In [ ]:
pronouns=['i','we','my','ours','us' ]
import re
count=0
for i in tokenize_text:
    if i.lower() in pronouns:
        count+=1
personal_pronouns=count
print('personal pronouns= ',personal_pronouns )

# AVG WORD LENGTH

In [ ]:
count=0
for i in tokenize_text:
    for j in i:
        count+=1
avg_word_length=count/len(tokenize_text)
print('avg word= ', avg_word_length)

In [ ]:
data={'positive_score':positive_score,'negative_score':negative_score,'Polarity_Score':Polarity_Score,'subjectiivity_score':subjectiivity_score,'avg_senetence_length':avg_senetence_length,'Percentage_of_Complex_words':Percentage_of_Complex_words,'Fog_Index':Fog_Index,'avg_no_of_words_per_sentence':avg_no_of_words_per_sentence,'complex_Word_Count':complex_Word_Count,'word_count':word_count,'syllable_count':syllable_count,'personal_pronouns':personal_pronouns,'avg_word_length':avg_word_length}

In [ ]:
#output=pd.DataFrame()
output=output.append(data,ignore_index=True)


In [ ]:
output.columns=['positive_score','negative_score','Polarity_Score','subjectiivity_score','avg_senetence_length','Percentage_of_Complex_words','Fog_Index','avg_no_of_words_per_sentence','complex_Word_Count','word_count','syllable_count','personal_pronouns','avg_word_length']
output

In [ ]:
with open('output.excel', 'a') as f:#creating text file 
    output.to_excel(f, index=False, header=False)
    